In [7]:
import numpy as np
import pandas as pd 
from opencage.geocoder import OpenCageGeocode
import geopy.distance

key = '09aadb1b1d8840acacfa0fcece0acb13'
geocoder = OpenCageGeocode(key)

class desafioBiodiversidade():
    def __init__(self,filePath):
        try:
            f = open(filePath,"r")
            self.originalData = f.readlines()
            self.data = [l.rstrip().split(";") for l in self.originalData]
            self.len = len(self.data)-1
            f.close()
        except:
            print("Error: cannot open file.")
            data = None  

    # Tópico 1 - Dados Faltantes
    def retornaMediaDadosFaltantesPorColuna(self,n):
        data = self.data
        
        nblank = [0]*len(data[0])
  
        for line in data:
            for ind,column in enumerate(line):
                if column == "Sem Informações" or column == "":
                    nblank[ind]+=1
 
        mean = [np.round((x/self.len)*100,2) for x in nblank]

        dict = {'Coluna': data[0], n+'-Faltantes':nblank, n+'-Média': mean}
        df = pd.DataFrame(dict)
        return df
    
    # Tópico 2 - Nível Taxonômico
    def retornaNivelTaxonomicoCadaOcorrencia(self):
        lista = self.data

        filo = []
        for i in range(len(lista[0])):
            
            if lista[0][i] == 'Especie':
              
                for k in range(1,len(lista)):
                    
                    if lista[k][i] != 'Sem Informações':
                        filo.append('{} - Especie'.format(k))
                    
                    elif lista[k][i-1] != 'Sem Informações':
                        filo.append('{} - Genero'.format(k))
                        
                    elif lista[k][i-2] != 'Sem Informações':
                        filo.append('{} - Familia'.format(k))
                        
                    elif lista[k][i-3] != 'Sem Informações':
                        filo.append('{} - Ordem'.format(k))
                    
                    elif lista[k][i-4] != 'Sem Informações':
                        filo.append('{} - Classe'.format(k))
                        
                    elif lista[k][i-5] != 'Sem Informações':
                        filo.append('{} - Filo'.format(k))
                        
                    else:
                        filo.append('{} - Reino'.format(k))
     
        return filo 

    # Tópico 3 - Filtro de Ocorrências
    def retonaSeOcorrenciaExiste(self, filterAmeaca):
        self.AmeacaList = []
        self.dataAux = []
        self.dataT = []
        
        # Matriz transposta
        for j in range(len(self.data[0])):        
            for i in range(len(self.data)-1):
                self.dataAux.append(self.data[i][j])
            self.dataT.append(self.dataAux)
            self.dataAux = []
        
        
        # Filtro estado de conservacao
        for i in range(len(self.dataT)):
            if self.dataT[i][0] == "Estado de conservacao":
                for j in range(len(self.dataT[i])):
                    if self.dataT[i][j] == filterAmeaca :
                        self.AmeacaList.append(["Amostra {}: {}: ".format(j, self.dataT[i-12][j]), filterAmeaca])
                    
        return self.AmeacaList
    
class GeoCode:

    def __init__(self,data):
        self.data = data
        self.topics = ["country","state","state_code","city"]

    def check_localization(self):
        indlat = self.data[0].index("Latitude")	
        indpais = self.data[0].index("Pais")
        
        print(len(self.data))
        result = []
        for ind,line in enumerate(self.data[1:]):
            print(ind)
            lat = self.parse_float(line[indlat])
            lon = self.parse_float(line[indlat+1])

            geo = geocoder.reverse_geocode(lat,lon)   # retorna info de lat,lon
            comp = geo[0]['components']               # separa info de localizacao
            info = self.get_info(comp) 

            res = self.info_compare(line[indpais:indpais+3],info)
    
            if not res:
                rlat,rlon = self.get_latlon(line[indpais:indpais+3])
                dist = self.get_distance((lat,lon),(rlat,rlon))
                result.append(dist)
            else:
                result.append(True)
                
        print("oi")
        return result



    # tenta a leitura de numeros float para latitude e longitude 
    def parse_float(self,info):
        try:
            value = float(info)
        except:
            value = 0.0
        return value


    # separa as informacoes de país, estado, código de estado e cidade
    def get_info(self,components):
        aux = []
        for elem in self.topics:
            try:
                value = components[elem]
            except:
                value = "Sem Informações"
            aux.append(value)
        return [aux[0],(aux[1],aux[2]),aux[3]]


    # compara as informacoes existentes
    def info_compare(self,line,info):

        correct = True
        for i, elem in enumerate(line):
            if line[i]!="Sem Informações" and info[i]!= "Sem Informações":
                if i==1:
                    if (line[i]!=info[i][0] and line[i]!=info[i][1]):
                        correct = False
                elif line[i]!=info[i]:
                    correct = False
        return correct 


    # busca a latitude e longitude de um endereco
    def get_latlon(self,line):

        address = self.concat_info(line) 
        geo = geocoder.geocode(address)
        lat,lon = geo[0]['geometry']['lat'], geo[0]['geometry']['lng']
        return lat,lon


    # concatena info em string para fazer a busca no geocode
    def concat_info(self,line):
        aux = ""
        for elem in line:
            if elem != "Sem Informações":
                aux += elem + ","
        return aux[:-1]


    # calcula distancia entre duas coordenadas
    def get_distance(self,coord1, coord2):
        return geopy.distance.geodesic(coord1,coord2).km
        
        
filePath1 = "portalbio_export_16-10-2019-14-39-54.csv"
filePath2 = "portalbio_export_17-10-2019-13-06-22.csv"
filePath3 = "portalbio_export_17-10-2019-13-15-00.csv"   
b1 = desafioBiodiversidade(filePath1)
b2 = desafioBiodiversidade(filePath2)
b3 = desafioBiodiversidade(filePath3)

# Análise Dados Faltantes

In [2]:
d1 = b1.retornaMediaDadosFaltantesPorColuna("1")

d2 = b2.retornaMediaDadosFaltantesPorColuna("2")
d1 = d1.join(d2.set_index('Coluna'), on='Coluna')
    
d3 = b3.retornaMediaDadosFaltantesPorColuna("3")    
d1 = d1.join(d3.set_index('Coluna'), on='Coluna')    

### Coluna com mais dados faltantes

In [3]:
print(d1[["Coluna","1-Média","2-Média","3-Média"]].to_string(index=False))

                              Coluna  1-Média  2-Média  3-Média
                 Nome da instituicao   100.00   100.00   100.00
                Sigla da instituicao     0.00     0.00     0.00
               Nome da base de dados     0.00     0.00     0.00
              Sigla da base de dados     0.00     0.00     0.00
           Responsavel pelo registro     0.08     0.00     0.01
        Numero do registro no portal     0.00     0.00     0.00
 Numero do registro na base de dados     6.01    19.28    27.71
                    Data do registro     3.50    19.28    26.17
                      Data do evento    54.45    12.25    37.26
                    Data de Carencia     0.00     0.00     0.00
                     Nome cientifico     1.69     1.80     4.87
                          Nome comum    74.21    40.54    68.44
    Nome cientifico na base de dados     0.00     0.00     0.00
                    Nivel taxonomico     1.69     1.80     4.87
                Numero de individuos    

### Média de dados faltantes por coluna nas três tabelas

In [4]:
tmean = d1[["Coluna","1-Média","2-Média","3-Média"]].mean(axis=1)
d1["Média Total"] = tmean
print(d1[["Coluna","1-Média","2-Média","3-Média","Média Total"]].to_string(index=False))

                              Coluna  1-Média  2-Média  3-Média  Média Total
                 Nome da instituicao   100.00   100.00   100.00   100.000000
                Sigla da instituicao     0.00     0.00     0.00     0.000000
               Nome da base de dados     0.00     0.00     0.00     0.000000
              Sigla da base de dados     0.00     0.00     0.00     0.000000
           Responsavel pelo registro     0.08     0.00     0.01     0.030000
        Numero do registro no portal     0.00     0.00     0.00     0.000000
 Numero do registro na base de dados     6.01    19.28    27.71    17.666667
                    Data do registro     3.50    19.28    26.17    16.316667
                      Data do evento    54.45    12.25    37.26    34.653333
                    Data de Carencia     0.00     0.00     0.00     0.000000
                     Nome cientifico     1.69     1.80     4.87     2.786667
                          Nome comum    74.21    40.54    68.44    61.063333

### Total de dados faltantes por tabela

In [5]:
print(d1[["Coluna","1-Faltantes","2-Faltantes","3-Faltantes"]].to_string(index=False))
tsum = d1[["Coluna","1-Faltantes","2-Faltantes","3-Faltantes"]].sum(axis=0)
print(tsum)

                              Coluna  1-Faltantes  2-Faltantes  3-Faltantes
                 Nome da instituicao         3912          555        58411
                Sigla da instituicao            0            0            0
               Nome da base de dados            0            0            0
              Sigla da base de dados            0            0            0
           Responsavel pelo registro            3            0            6
        Numero do registro no portal            0            0            0
 Numero do registro na base de dados          235          107        16183
                    Data do registro          137          107        15287
                      Data do evento         2130           68        21764
                    Data de Carencia            0            0            0
                     Nome cientifico           66           10         2843
                          Nome comum         2903          225        39975
    Nome cie

# Análise Nível Taxonômico

In [6]:
nt1 = b1.retornaNivelTaxonomicoCadaOcorrencia()
nt2 = b2.retornaNivelTaxonomicoCadaOcorrencia()
nt3 = b3.retornaNivelTaxonomicoCadaOcorrencia()

### Nível taxonômico por tabela 

In [7]:
level = ["Reino","Filo","Classe","Ordem","Familia","Genero","Especie"]

# Retorna quantidade de linhas da tabela
# e uma lista de contadores dos níveis taxonômicos
def count_per_level(nt):    
    c = [0]*7
    for line in nt:
        c[level.index(line.split("- ")[1])] += 1
    return c

In [8]:
c1 =count_per_level(nt1)
c2 =count_per_level(nt2)
c3 =count_per_level(nt3)

# Calcula a média
mc1 = [np.round((x/b1.len)*100,2) for x in c1]
mc2 = [np.round((x/b2.len)*100,2) for x in c2]
mc3 = [np.round((x/b3.len)*100,2) for x in c3]

# Converte em dicionário e depois Dataframe 
dict = {'Nível':level, "Tabela1":c1, "Média1(%)":mc1 ,"Tabela2":c2, "Média2(%)":mc2, "Tabela3":c3, "Média3(%)":mc3}
nt = pd.DataFrame(dict)

### Porcentagem de níveis por tabela

In [9]:
print(nt[["Nível","Média1(%)","Média2(%)","Média3(%)"]].to_string(index=False))

   Nível  Média1(%)  Média2(%)  Média3(%)
   Reino       3.17       2.52       5.34
    Filo       0.59       0.36       0.56
  Classe       0.54       4.14       0.74
   Ordem       0.43       1.26       1.45
 Familia      11.78       3.42       7.16
  Genero      20.94       3.06       9.64
 Especie      62.55      85.23      75.11


# Análise filtro de ocorrências - Estado de conservação

In [28]:
estado = ["Espécie Ameaçada", "Espécie não Ameaçada"]

Aec1 = b1.retonaSeOcorrenciaExiste(estado[0])
Nec1 = b1.retonaSeOcorrenciaExiste(estado[1])

Aec2 = b2.retonaSeOcorrenciaExiste(estado[0])
Nec2 = b2.retonaSeOcorrenciaExiste(estado[1])

Aec3 = b3.retonaSeOcorrenciaExiste(estado[0])
Nec3 = b3.retonaSeOcorrenciaExiste(estado[1])

In [39]:
# Converte em dicionário e depois Dataframe 
dict = {'Estado de Conservação':estado, "Tabela1":[len(Aec1),len(Nec1)], "Média1(%)":[np.round((len(Aec1)/b1.len)*100,2),np.round((len(Nec1)/b1.len)*100,2)],\
                                        "Tabela2":[len(Aec2),len(Nec2)], "Média2(%)":[np.round((len(Aec2)/b2.len)*100,2),np.round((len(Nec2)/b2.len)*100,2)],\
                                        "Tabela3":[len(Aec3),len(Nec3)], "Média3(%)":[np.round((len(Aec3)/b3.len)*100,2),np.round((len(Nec3)/b3.len)*100,2)]}
ec = pd.DataFrame(dict)

### Número total de ocorrências da coluna "Estado de conservação"

In [40]:
print(ec[["Estado de Conservação","Tabela1","Tabela2","Tabela3"]].to_string(index=False))

Estado de Conservação  Tabela1  Tabela2  Tabela3
     Espécie Ameaçada      570       18     6203
 Espécie não Ameaçada     3329      536    52200


### Porcentagem de ocorrências

In [41]:
print(ec[["Estado de Conservação","Média1(%)","Média2(%)","Média3(%)"]].to_string(index=False))

Estado de Conservação  Média1(%)  Média2(%)  Média3(%)
     Espécie Ameaçada      14.57       3.24      10.62
 Espécie não Ameaçada      85.10      96.58      89.37


# Análise da Localização

In [8]:
gCode = GeoCode(b1.data)
result = gCode.check_localization()
print(result)

3913
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
2

RateLimitExceededError: Your rate limit has expired. It will reset to 2500 on 2019-11-04T00:00:00